<a href="https://colab.research.google.com/github/guylozius/sales-predictions/blob/main/sales_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The goal

<p> The goal of this is to help the retailer understand the properties of products and outlets that play crucial roles in increasing sales.<p>

# Importing the dataframe

In [315]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/coding dojo/stack_one/sales_predictions.csv")
df.head(10)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


#Cleaning the dataframe 

## Show the size of the dataframe.

In [316]:
df.shape

(8523, 12)

## Show the data type of each column.

In [317]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


## Checking if any rows are duplicated and then droping duplicated rows.

In [318]:
df.duplicated().sum()

0

## Looking for missing value

In [319]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [320]:
df.isna().sum().sum()

3873

> The dataframe is missing values in Item_Weight and in Outlet_Size




## Dropping Item_Weight

In [321]:
#Seeing how many unique values in this columns
df['Item_Weight'].value_counts().sum()

7060

Through my Observation I deceid to drop Item_Weight becasuse it had too many unique value for me to find any type pattern to replace missing  or impute it ethier, and they too rows for me delete it would to much good data to lose.

In [322]:
df.drop(columns='Item_Weight', inplace=True)

> Item_Weight columns have too many unique values

## Finding pattern for Outlet_Size

In [323]:
#seeing how many unique values in this columns
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

> Outlet_Size columns only have three unique values

In [324]:
#Seeing how many unique values in this columns
df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [325]:
#Returing the number of Nan in Outlet_Size group by Outlet_Type & Outlet_Location_Type
df.groupby(['Outlet_Type','Outlet_Location_Type'])['Outlet_Size'].apply(lambda x: x.isnull().sum())

Outlet_Type        Outlet_Location_Type
Grocery Store      Tier 1                     0
                   Tier 3                   555
Supermarket Type1  Tier 1                     0
                   Tier 2                  1855
                   Tier 3                     0
Supermarket Type2  Tier 3                     0
Supermarket Type3  Tier 3                     0
Name: Outlet_Size, dtype: int64

> Only Tier 3 Grocery Store and Supermarket Type1 Tier 2 have missing values

In [326]:
#returning non-missing of Outlet_Size group by Outlet_Type & Outlet_Location_Type
df.groupby(['Outlet_Type','Outlet_Location_Type'])['Outlet_Size'].count()

Outlet_Type        Outlet_Location_Type
Grocery Store      Tier 1                   528
                   Tier 3                     0
Supermarket Type1  Tier 1                  1860
                   Tier 2                   930
                   Tier 3                   932
Supermarket Type2  Tier 3                   928
Supermarket Type3  Tier 3                   935
Name: Outlet_Size, dtype: int64

> It seem that Tier 3 Grocery Store do not have non-missing values, but Tier 2 Supermarket Type1 do, thus I look for pattern there.

In [327]:
#Making Filter for  Tier 2 Supermarket Type1
Tier_2 = df['Outlet_Location_Type'] == 'Tier 2'
Supermarket_Type1	= df['Outlet_Type'] == 'Supermarket Type1'


In [328]:
#Return data frame with only Tier 2 Supermarket Type1
df.loc[Tier_2 & Supermarket_Type1, :].value_counts()

Item_Identifier  Item_Fat_Content  Item_Visibility  Item_Type              Item_MRP  Outlet_Identifier  Outlet_Establishment_Year  Outlet_Size  Outlet_Location_Type  Outlet_Type        Item_Outlet_Sales
DRA12            LF                0.000000         Soft Drinks            141.9154  OUT035             2004                       Small        Tier 2                Supermarket Type1  992.7078             1
FDU49            Regular           0.030689         Canned                 85.7540   OUT035             2004                       Small        Tier 2                Supermarket Type1  2163.8500            1
FDU26            Regular           0.042611         Dairy                  117.2782  OUT035             2004                       Small        Tier 2                Supermarket Type1  715.0692             1
FDU31            Regular           0.024986         Fruits and Vegetables  216.7508  OUT035             2004                       Small        Tier 2                Superma

In [329]:
# See the value of non-missing value of Tier_2 Supermarket_Type1
df.loc[Tier_2 & Supermarket_Type1, :]['Outlet_Size'].value_counts()

Small    930
Name: Outlet_Size, dtype: int64

I see that non-missing value Tier 2 Supermarket Type1 are all small

In [330]:
# Making filter for Tier 1 Grocery Stores
Tier_1 = df['Outlet_Location_Type'] == 'Tier 1'
Grocery_Store	= df['Outlet_Type'] == 'Grocery Store'


In [331]:
#Return data frame with only Tier 1 Grocery Stores
df.loc[Tier_1 & Grocery_Store, :].tail(10)

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
8390,NCQ05,Low Fat,0.037829,Health and Hygiene,151.0708,OUT019,1985,Small,Tier 1,Grocery Store,451.4124
8404,DRH39,low fat,0.000000,Dairy,75.9670,OUT019,1985,Small,Tier 1,Grocery Store,153.1340
8405,FDB09,Low Fat,0.100493,Fruits and Vegetables,123.1046,OUT019,1985,Small,Tier 1,Grocery Store,124.5046
8442,FDX40,Low Fat,0.173324,Frozen Foods,39.9164,OUT019,1985,Small,Tier 1,Grocery Store,38.6164
8449,FDG02,Low Fat,0.019717,Canned,191.9188,OUT019,1985,Small,Tier 1,Grocery Store,571.2564
8454,NCH54,Low Fat,0.127234,Household,158.3920,OUT019,1985,Small,Tier 1,Grocery Store,159.7920
8458,FDX20,Low Fat,0.074518,Fruits and Vegetables,227.3720,OUT019,1985,Small,Tier 1,Grocery Store,452.7440
8469,FDQ45,Regular,0.019114,Snack Foods,182.1608,OUT019,1985,Small,Tier 1,Grocery Store,367.5216
8480,FDQ58,Low Fat,0.000000,Snack Foods,154.5340,OUT019,1985,Small,Tier 1,Grocery Store,459.4020
8490,FDU44,Regular,0.102296,Fruits and Vegetables,162.3552,OUT019,1985,Small,Tier 1,Grocery Store,487.3656


In [332]:
# See the value of non-missing value of Tier_1 Grocery_Store
df.loc[Tier_1 & Grocery_Store, :]['Outlet_Size'].value_counts()

Small    528
Name: Outlet_Size, dtype: int64

## Replacing missing values

Through my Observation I decide to replace my missing data in Outlet_size with the value 'Small' because of small of amount unique data I was able to pattern with Tier_2 Supermarket_Type1 that was missing data with ones that was not missing data and one that one that was not missing data thier value was 'Small'
, so it safe to assume that other was same. When came to Grocery_Store missing data I was able to see other was given the value 'Small' as well and assume that missing data's value would be 'Small'. I colud not delete row because it would to much good data lost.

In [333]:
df['Outlet_Size'] = df['Outlet_Size'].fillna('Small')

## Making sure there is no more missing value

In [334]:
df.isna().sum()

Item_Identifier              0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [335]:
df.groupby(['Outlet_Type','Outlet_Location_Type'])['Outlet_Size'].count()

Outlet_Type        Outlet_Location_Type
Grocery Store      Tier 1                   528
                   Tier 3                   555
Supermarket Type1  Tier 1                  1860
                   Tier 2                  2785
                   Tier 3                   932
Supermarket Type2  Tier 3                   928
Supermarket Type3  Tier 3                   935
Name: Outlet_Size, dtype: int64

## looking for any inconsistent categories of data 

In [336]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Fat_Content           8523 non-null   object 
 2   Item_Visibility            8523 non-null   float64
 3   Item_Type                  8523 non-null   object 
 4   Item_MRP                   8523 non-null   float64
 5   Outlet_Identifier          8523 non-null   object 
 6   Outlet_Establishment_Year  8523 non-null   int64  
 7   Outlet_Size                8523 non-null   object 
 8   Outlet_Location_Type       8523 non-null   object 
 9   Outlet_Type                8523 non-null   object 
 10  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory usage: 732.6+ KB


In [337]:
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [338]:
df['Item_Fat_Content'] = df.loc[:, 'Item_Fat_Content'].str.replace('low fat', 'Low Fat').replace('LF','Low Fat').replace('reg','Regular')

In [339]:
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [340]:
df['Outlet_Size'].unique()

array(['Medium', 'Small', 'High'], dtype=object)

In [341]:
df['Outlet_Size'] = df['Outlet_Size'].str.replace('Large','High')

## For any numerical columns, obtain the summary statistics of each (min, max, mean)

In [342]:
df[['Item_Outlet_Sales']].describe().round(2)

,Item_Outlet_Sales
count,8523.00
mean,2181.29
std,1706.50
min,33.29
25%,834.25
50%,1794.33
75%,3101.30
max,13086.96


- min 33.29
- max 13086.96
- mean 2181.29

In [343]:
df[['Item_Visibility']].describe().round(2)

,Item_Visibility
count,8523.00
mean,0.07
std,0.05
min,0.00
25%,0.03
50%,0.05
75%,0.09
max,0.33


- min 0.00
- max 0.33
- mean 0.07

In [344]:
df[['Item_MRP']].describe().round(2)

,Item_MRP
count,8523.00
mean,140.99
std,62.28
min,31.29
25%,93.83
50%,143.01
75%,185.64
max,266.89


- min 31.29
- max 266.89
- mean 140.99

In [345]:
df[['Outlet_Establishment_Year']].describe().round()

,Outlet_Establishment_Year
count,8523.0
mean,1998.0
std,8.0
min,1985.0
25%,1987.0
50%,1999.0
75%,2004.0
max,2009.0


- min 1985
- max 2009
- mean 1998